In [1]:
!nvidia-smi

Thu May 13 22:46:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-bj4rwjul
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-bj4rwjul
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=208f21d67c6ba09f7b323b77a4daecdd4a2f2aa41b2b774f88d4d2e95ac08687
  Stored in directory: /tmp/pip-ephem-wheel-cache-nh97otvq/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [7]:
%%cu
#include <iostream>
#include <cstdlib>

#define N (100)
#define BLOCKS 10

using namespace std;

__global__ void dot_product(int *a, int *b, int *c)
{
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid < N)
      c[tid] = a[tid] * b[tid];
}

int main()
{
    int a[N], b[N], c[N];
    int *d_a, *d_b, *d_c;
    int size = sizeof(int);
    int sum = 0;
 
    cudaMalloc((void **)&d_a, N*size);
    cudaMalloc((void **)&d_b, N*size);
    cudaMalloc((void **)&d_c, N*size);
 
    for (int i=0; i<N; i++) {
        a[i] = 1;
        b[i] = 1;
    }
 
    cudaMemcpy(d_a, &a, N*size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, &b, N*size, cudaMemcpyHostToDevice);
    
    dot_product<<<BLOCKS, BLOCKS>>>(d_a, d_b, d_c);

    cudaMemcpy(&c, d_c, N*size, cudaMemcpyDeviceToHost);
 
    for (int i=0; i<N; i++) {
        sum += c[i];
    }
 
    printf("Dot Product: %d\n", sum);
 
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
 
    return 0;
}

Dot Product: 100

